# Ch11-3 K-means

(1) k개의 중심점 설정 : 임의로 k 개로 잡는다.  
(2) 샘플 할당 :  각 데이터 포인트가 어느 중심점에 가까운지에 따라 할당한다.  
(3) 중심 업데이트 : 할당된 데이터들을 기반으로 중심점을 업데이트 한다.
(4) 샘플 재할당 : 중심이 이동하고 보니 더 가까운 중심점이 달라지므로 그에 따라 새로 재할당한다.  
(5) 샘플 재할당에 따른 군집 업데이트  
(6) 중심점 업데이트 : 더 이상 업데이트가 되지 않을 때 종료  

## K-평균 군집화의 장단점
- 장점  
    1. 계층군집화의 계산량에 비해 훨씬 적다. (계층군집화는 $O(n^3)$ 꼴)
    2. 군집 개수 설정에 제약이 없고 쉽다. 그래서 가장 많이, 널리 쓰인다.

- 단점  
    1. 초기 중심 설정(랜덤)에 따른 수행할 때마다 다른 결과를 낼 가능성 즉 임의성이 존재한다.
    2. 데이터 분포가 특이하거나 군집별 밀도 차이가 존재하면 좋은 성능을 내기 어렵다. 

        보통 군집이 원형으로 잡히는데, 모양이 이상한 군집이라던가 밀도가 차이나면 성능이 떨어진다.

    3. 유클리디안 거리만 사용해야한다. 중심점이 유클리드 기하의 개념이기 때문.
    4. 수렴하지 않을 가능성이 있다. 중심점이 계속해서 업데이트가 되기도 하고, 그만큼 수렴하지 않을 가능성도 존재한다. 그래서 `max_iter` 키워드가 존재한다.


## sklearn.cluster.KMeans

- 주요 입력
    - `n_clusters` : 군집 개수
    - `max_iter` : 최대 이터레이션 횟수 (크게 잡아도 오래걸리지는 않음)
- 주요 메서드
    - `fit(X)` : 데이터 X에 대한 군집화 모델 학습
    - `fit_predict(X)` : 데이터 X에 대한 군집화 모델 학습 및 라벨 반환
- 주요 속성
    - `labels_` : fitting한 데이터에 있는 샘플들이 속한 군집 정보 (ndarray로 반환)
    - `cluster_centers_` :  fitting한 데이터에 있는 샘플들이 속한 군집 중심점 (ndarray로 반환)
        
        엄밀히 얘기하면 계층적 군집화에는 중심점이라는 개념이 없다. 중심을 가지고 군집화가 이루어지는 것이 아니기 때문. 하지만 k-means는 처음부터 중심점이 있다. 그래서 중심점에 대한 정보를 계층적 군집화에 비해 쉽게 알 수 있다는 장점이 있다.

In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/2. 탐색적 데이터 분석/데이터")

print(pd.__version__)
print(np.__version__)

1.4.1
1.22.4


### 고객 정보 데이터 

In [2]:
df = pd.read_csv("Telco_customer_info.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [3]:
df.set_index('customerID', inplace = True)
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
customerID,,,,,,,,,,,,,,,,,,,
7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


### 범주형 변수 더미화

In [4]:
df = pd.get_dummies(df, drop_first = True)
df.head()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
customerID,,,,,,,,,,,,,,,,,,,,,
7590-VHVEG,0,1,29.85,29.85,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
5575-GNVDE,0,34,56.95,1889.50,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,1
3668-QPYBK,0,2,53.85,108.15,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,1
7795-CFOCW,0,45,42.30,1840.75,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
9237-HQITU,0,2,70.70,151.65,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0


In [5]:
df.columns

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'Contract_One year', 'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'],
      dtype='object')

### k-means 수행

모델을 인스턴스화 시키고 fitting을 진행한다.  
그리고 피팅한 정보, 즉 학습시킨 모델을 clusters라고 저장.  

In [7]:
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters = 3, random_state = 2022).fit(df) # 결과 X, 모델 O

### 군집 중심, 특성 확인. 
특성을 확인하기 위해 cluster_centers_를 중심으로 본다.  
중심 정보 (이름이 없기 때문에 컬럼과 인덱스 부여)

In [8]:
# 군집 중심 확인 및 이름 붙이기
pd.DataFrame(clusters.cluster_centers_, 
             columns = df.columns,
             index = range(3))

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0.207325,44.132216,77.818963,3281.164184,0.517691,0.556797,0.303538,0.860335,0.139665,0.497827,...,1.241465e-03,0.545003,1.241465e-03,0.553073,0.301676,0.214773,0.671633,0.260708,0.347610,0.116077
1,0.128661,18.264522,49.767331,688.070319,0.501200,0.376140,0.285166,0.891263,0.108737,0.269803,...,3.643783e-01,0.194911,3.643783e-01,0.195391,0.145943,0.165627,0.526644,0.164186,0.346375,0.324532
2,0.216733,64.384861,97.979243,6297.778685,0.499602,0.740239,0.336255,0.998406,0.001594,0.829482,...,8.049117e-16,0.807171,8.049117e-16,0.817530,0.301195,0.517131,0.710757,0.332271,0.288446,0.051793


### 해석

1번 그룹은 상대적으로 고령 고객이 적고, 2번 그룹은 월별 지불액이 높다.  
2번 그룹은 폰서비스를 가장 많이 사용하며, 다회선 사용률이 가장 높다.  

